In [1]:
import pandas as pd
import numpy as np
import re
from catboost import CatBoostRegressor
from sklearn.ensemble import RandomForestRegressor

In [32]:
df_train = pd.read_excel('data/Test_set.xlsx')

In [33]:
df_train.head(10)

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info
0,Jet Airways,6/06/2019,Delhi,Cochin,DEL → BOM → COK,17:30,04:25 07 Jun,10h 55m,1 stop,No info
1,IndiGo,12/05/2019,Kolkata,Banglore,CCU → MAA → BLR,06:20,10:20,4h,1 stop,No info
2,Jet Airways,21/05/2019,Delhi,Cochin,DEL → BOM → COK,19:15,19:00 22 May,23h 45m,1 stop,In-flight meal not included
3,Multiple carriers,21/05/2019,Delhi,Cochin,DEL → BOM → COK,08:00,21:00,13h,1 stop,No info
4,Air Asia,24/06/2019,Banglore,Delhi,BLR → DEL,23:55,02:45 25 Jun,2h 50m,non-stop,No info
5,Jet Airways,12/06/2019,Delhi,Cochin,DEL → BOM → COK,18:15,12:35 13 Jun,18h 20m,1 stop,In-flight meal not included
6,Air India,12/03/2019,Banglore,New Delhi,BLR → TRV → DEL,07:30,22:35,15h 5m,1 stop,No info
7,IndiGo,1/05/2019,Kolkata,Banglore,CCU → HYD → BLR,15:15,20:30,5h 15m,1 stop,No info
8,IndiGo,15/03/2019,Kolkata,Banglore,CCU → BLR,10:10,12:55,2h 45m,non-stop,No info
9,Jet Airways,18/05/2019,Kolkata,Banglore,CCU → BOM → BLR,16:30,22:35,6h 5m,1 stop,No info


In [34]:
df_train['Destination'].replace({'New Delhi' : 'Delhi'}, inplace = True)
df_train['Source'].replace({'New Delhi' : 'Delhi'}, inplace = True)

In [35]:
df_train.Total_Stops.fillna('non-stop', inplace = True)

In [36]:
df_train['Total_Stops'].replace({'non-stop' : '0 stop'}, inplace = True)

In [37]:
df_train.Route.fillna('BLR → DEL', inplace = True)

In [38]:
df_train['Additional_Info'].replace({'No info' : 'No Info'}, inplace = True)

In [39]:
from sklearn.preprocessing import LabelEncoder

In [10]:
la = LabelEncoder()
la.fit(df_train.Airline)


In [40]:
df_train.Airline = la.transform(df_train.Airline)

In [41]:
lc = LabelEncoder()
lc.fit(['Banglore', 'Chennai', 'Delhi', 'Kolkata', 'Cochin', 'Hyderabad', 'Mumbai'])
df_train.Source = lc.transform(df_train.Source)
df_train.Destination = lc.transform(df_train.Destination)

In [42]:
lts = LabelEncoder()
lts.fit(['0 stop', '1 stop', '2 stops', '3 stops', '4 stops', '5 stops'])
df_train.Total_Stops = lts.transform(df_train.Total_Stops)

In [13]:
lai = LabelEncoder()
lai.fit(df_train.Additional_Info)


In [43]:
df_train.Additional_Info = lai.transform(df_train.Additional_Info)

In [14]:
df_train.Route = df_train.Route.str.split(' → ')

In [15]:
df_train.Route[0][0]

'BLR'

In [16]:
city_code = set()
for i in range(len(df_train)):
    del df_train.Route[i][0]
    del df_train.Route[i][-1]
    for j in range(len(df_train.Route[i])):
        city_code.add(df_train.Route[i][j])

In [17]:
len(city_code)

42

In [58]:
df_train.isnull().sum()

Airline            0
Date_of_Journey    0
Source             0
Destination        0
Route              0
Dep_Time           0
Arrival_Time       0
Duration           0
Total_Stops        0
Additional_Info    0
dtype: int64

In [59]:
len(df_train)

2671

In [60]:
len(hour)

2671

In [71]:
hour = df_train.Duration.str.split('h')
hour.tail(20)
#for i in range (2650):
#    if((hour[i][1]) == ''):
#        print(hour[i][0])

2651    [12,  30m]
2652    [14,  20m]
2653      [6,  5m]
2654    [24,  40m]
2655     [9,  20m]
2656    [12,  10m]
2657    [14,  20m]
2658    [21,  10m]
2659      [7,  5m]
2660          [5m]
2661    [33,  15m]
2662     [1,  30m]
2663     [8,  15m]
2664    [10,  15m]
2665     [1,  30m]
2666    [23,  55m]
2667     [2,  35m]
2668     [6,  35m]
2669    [15,  15m]
2670    [14,  20m]
Name: Duration, dtype: object

In [72]:
df_train['Duration'].replace({'5m' : '0h 5m'}, inplace=True)

In [73]:
hour = df_train.Duration.str.split('h')
for i in range (len(hour)):
    if((hour[i][1]) == ''):
        hour[i][1] = ' 0m'

In [74]:
for i in range (len(hour)):
    hour[i][0] = int(hour[i][0])*60
    hour[i][1] = int(hour[i][1].split('m')[0])
    hour[i][0] = hour[i][0] + hour[i][1]

In [75]:
for i in range (len(hour)):
    hour[i] = hour[i][0]
df_train.Duration = hour.astype(int)

In [76]:
dep = df_train.Dep_Time.str.split(':')
for i in range(len(dep)):
    dep[i] = dep[i][0] + dep[i][1]
df_train.Dep_Time = dep.astype(int)

In [77]:
arr = df_train.Arrival_Time.str.split(' ')
for i in range (len(arr)):
    arr[i] = arr[i][0]
arr = arr.str.split(':')
for i in range(len(arr)):
    arr[i] = arr[i][0] + arr[i][1]
df_train.Arrival_Time = arr.astype(int)

In [78]:
def add_datepart(df, fldname, drop=True, time=False):
    "Helper function that adds columns relevant to a date."
    fld = df[fldname]
    fld_dtype = fld.dtype
    if isinstance(fld_dtype, pd.core.dtypes.dtypes.DatetimeTZDtype):
        fld_dtype = np.datetime64

    if not np.issubdtype(fld_dtype, np.datetime64):
        df[fldname] = fld = pd.to_datetime(fld, infer_datetime_format=True)
    targ_pre = re.sub('[Dd]ate$', '', fldname)
    attr = ['Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear',
            'Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start']
    if time: attr = attr + ['Hour', 'Minute', 'Second']
    for n in attr: df[targ_pre + n] = getattr(fld.dt, n.lower())
    df[targ_pre + 'Elapsed'] = fld.astype(np.int64) // 10 ** 9
    if drop: df.drop(fldname, axis=1, inplace=True)


In [79]:
add_datepart(df_train, "Date_of_Journey")

In [80]:
df_train.drop(columns='Route', inplace=True)

In [218]:
#df_train_tmp = df_train

In [219]:
#df_train_con = pd.concat([df_train_tmp,pd.DataFrame(columns=list(city_code))], sort=False)

In [220]:
#df_train_con.fillna(value=False, inplace=True)

In [ ]:
#for i in range (len(df_train_con)):
#    for j in range (len(df_train_con.Route[i])):
#        x = df_train_con.Route[i][j]
#        df_train_con[x][i] = True

In [209]:
#df_train_con['IXR'].head()

0    False
1     True
2    False
3    False
4    False
Name: IXR, dtype: object

In [81]:
df_train.head(10)

,Airline,Source,Destination,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Date_of_JourneyYear,Date_of_JourneyMonth,...,Date_of_JourneyDay,Date_of_JourneyDayofweek,Date_of_JourneyDayofyear,Date_of_JourneyIs_month_end,Date_of_JourneyIs_month_start,Date_of_JourneyIs_quarter_end,Date_of_JourneyIs_quarter_start,Date_of_JourneyIs_year_end,Date_of_JourneyIs_year_start,Date_of_JourneyElapsed
0,4,3,2,1730,425,655,1,6,2019,6,...,6,3,157,False,False,False,False,False,False,1559779200
1,3,5,0,620,1020,240,1,6,2019,12,...,5,3,339,False,False,False,False,False,False,1575504000
2,4,3,2,1915,1900,1425,1,5,2019,5,...,21,1,141,False,False,False,False,False,False,1558396800
3,6,3,2,800,2100,780,1,6,2019,5,...,21,1,141,False,False,False,False,False,False,1558396800
4,0,0,3,2355,245,170,0,6,2019,6,...,24,0,175,False,False,False,False,False,False,1561334400
5,4,3,2,1815,1235,1100,1,5,2019,12,...,6,4,340,False,False,False,False,False,False,1575590400
6,1,0,3,730,2235,905,1,6,2019,12,...,3,1,337,False,False,False,False,False,False,1575331200
7,3,5,0,1515,2030,315,1,6,2019,1,...,5,5,5,False,False,False,False,False,False,1546646400
8,3,5,0,1010,1255,165,0,6,2019,3,...,15,4,74,False,False,False,False,False,False,1552608000
9,4,5,0,1630,2235,365,1,6,2019,5,...,18,5,138,False,False,False,False,False,False,1558137600


In [82]:
df_train.dtypes

Airline                            int64
Source                             int64
Destination                        int64
Dep_Time                           int64
Arrival_Time                       int64
Duration                           int64
Total_Stops                        int64
Additional_Info                    int64
Date_of_JourneyYear                int64
Date_of_JourneyMonth               int64
Date_of_JourneyWeek                int64
Date_of_JourneyDay                 int64
Date_of_JourneyDayofweek           int64
Date_of_JourneyDayofyear           int64
Date_of_JourneyIs_month_end         bool
Date_of_JourneyIs_month_start       bool
Date_of_JourneyIs_quarter_end       bool
Date_of_JourneyIs_quarter_start     bool
Date_of_JourneyIs_year_end          bool
Date_of_JourneyIs_year_start        bool
Date_of_JourneyElapsed             int64
dtype: object

In [28]:
df_train['Price'] = np.log(df_train.Price + 1)

In [30]:
train = df_train

In [83]:
test = df_train

In [304]:
from sklearn.model_selection import train_test_split

In [315]:
def rmsle(y,y0):
    return np.sqrt(np.mean(np.square(np.log1p(y) - np.log1p(y0))))


In [31]:
model = CatBoostRegressor(iterations=500, learning_rate=0.2, depth = 8, loss_function='RMSE')
model.fit(train.drop(columns = ['Price'], axis=1), train.Price, )

0:	learn: 7.2289200	total: 703ms	remaining: 5m 50s
1:	learn: 5.7926506	total: 713ms	remaining: 2m 57s
2:	learn: 4.6460335	total: 1.11s	remaining: 3m 4s
3:	learn: 3.7390627	total: 1.17s	remaining: 2m 24s
4:	learn: 3.0040346	total: 1.19s	remaining: 1m 57s
5:	learn: 2.4165167	total: 1.22s	remaining: 1m 40s
6:	learn: 1.9471013	total: 1.24s	remaining: 1m 27s
7:	learn: 1.5716663	total: 1.25s	remaining: 1m 16s
8:	learn: 1.2716626	total: 1.27s	remaining: 1m 9s
9:	learn: 1.0360915	total: 1.28s	remaining: 1m 2s
10:	learn: 0.8463565	total: 1.29s	remaining: 57.5s
11:	learn: 0.7009873	total: 1.31s	remaining: 53.4s
12:	learn: 0.5887331	total: 1.32s	remaining: 49.5s
13:	learn: 0.5012938	total: 1.33s	remaining: 46.3s
14:	learn: 0.4370548	total: 1.35s	remaining: 43.8s
15:	learn: 0.3865758	total: 1.37s	remaining: 41.4s
16:	learn: 0.3503038	total: 1.38s	remaining: 39.1s
17:	learn: 0.3255532	total: 1.38s	remaining: 37s
18:	learn: 0.2948124	total: 1.4s	remaining: 35.5s
19:	learn: 0.2809074	total: 1.43s	rem

188:	learn: 0.2155844	total: 2.53s	remaining: 4.16s
189:	learn: 0.2155844	total: 2.53s	remaining: 4.13s
190:	learn: 0.2155842	total: 2.54s	remaining: 4.11s
191:	learn: 0.2155840	total: 2.55s	remaining: 4.08s
192:	learn: 0.2155838	total: 2.55s	remaining: 4.06s
193:	learn: 0.2155838	total: 2.56s	remaining: 4.04s
194:	learn: 0.2155836	total: 2.56s	remaining: 4.01s
195:	learn: 0.2155835	total: 2.57s	remaining: 3.98s
196:	learn: 0.2155833	total: 2.57s	remaining: 3.96s
197:	learn: 0.2155832	total: 2.58s	remaining: 3.93s
198:	learn: 0.2155831	total: 2.58s	remaining: 3.9s
199:	learn: 0.2155830	total: 2.58s	remaining: 3.88s
200:	learn: 0.2155829	total: 2.59s	remaining: 3.85s
201:	learn: 0.2155828	total: 2.59s	remaining: 3.83s
202:	learn: 0.2155827	total: 2.6s	remaining: 3.8s
203:	learn: 0.2155826	total: 2.6s	remaining: 3.77s
204:	learn: 0.2155826	total: 2.6s	remaining: 3.75s
205:	learn: 0.2155825	total: 2.61s	remaining: 3.72s
206:	learn: 0.2155825	total: 2.61s	remaining: 3.7s
207:	learn: 0.2155

365:	learn: 0.2154344	total: 3.3s	remaining: 1.21s
366:	learn: 0.2154344	total: 3.31s	remaining: 1.2s
367:	learn: 0.2154344	total: 3.31s	remaining: 1.19s
368:	learn: 0.2154344	total: 3.31s	remaining: 1.18s
369:	learn: 0.2154344	total: 3.32s	remaining: 1.17s
370:	learn: 0.2154344	total: 3.33s	remaining: 1.16s
371:	learn: 0.2154344	total: 3.34s	remaining: 1.15s
372:	learn: 0.2154344	total: 3.35s	remaining: 1.14s
373:	learn: 0.2154344	total: 3.35s	remaining: 1.13s
374:	learn: 0.2154344	total: 3.35s	remaining: 1.12s
375:	learn: 0.2154344	total: 3.36s	remaining: 1.11s
376:	learn: 0.2154344	total: 3.36s	remaining: 1.1s
377:	learn: 0.2154344	total: 3.36s	remaining: 1.08s
378:	learn: 0.2154344	total: 3.37s	remaining: 1.07s
379:	learn: 0.2154344	total: 3.37s	remaining: 1.06s
380:	learn: 0.2154344	total: 3.37s	remaining: 1.05s
381:	learn: 0.2154344	total: 3.38s	remaining: 1.04s
382:	learn: 0.2154344	total: 3.38s	remaining: 1.03s
383:	learn: 0.2154344	total: 3.38s	remaining: 1.02s
384:	learn: 0.2

In [84]:
pred = model.predict(test)

In [85]:
mp = np.exp(pred) + 1

In [86]:
mp

array([13162.12205488,  5319.54956454, 11461.13884862, ...,
       12018.08039934,  8275.5818062 , 10055.37210344])

In [89]:
res = pd.DataFrame(data=mp)

In [94]:
res.to_excel('pred.xlsx', index=False)